# Universality: Working with one qubit

In quantum computing, we have two fundamental structures: states and operators, which we can represent with vectors and matrices respectively. Vectors must have norm 1 (since they determine a sum of probabilities) and matrices must be unitary, i.e. $U^{-1} = U^*$ (to preserve the norm of the vectors). When building a quantum computer, we would like it to be able to generate any $U$ operator we need. However, we cannot physically implement every possible operator, so the goal is to create subsets of gates or operators that are able to generate all the others.

It can be easily proved that in the case of one qubit, we can express any single-qubit unitary $U$ in terms of rotations as follows: $e^{i\phi} R_Z(\gamma) R_X(\beta) R_Z(\alpha)$. This would mean that $R_X$ and $R_Z$ form a universal set. That is why, in this challenge we will be asked to calculate the parameters $\alpha$, $\beta$, and $\gamma$ of a given gate $U$. In particular, we will use a variational method to find these parameters so we will also ask for an error function that tells us how well we are approximating our matrix to $U$.

*Note: Remember that if we write a gate $U$ as $A\cdot B$, it means that $B$ is executed first, and then $A$.*

## Challenge code

You must complete the `get_matrix` function below in which you will return the matrix associated with those parameters. Once this is done, a small script will be given to optimize the algorithm looking for the best parameters. In order to do this you must also define the `error` function that determines how well you are approximating your matrix to the target operator.

### Input

As input to this problem, you are given a $2\times2$ complex matrix that you will try to approximate.

### Output

This code will calculate a `list(float)` containing the four parameters ($\phi$, $\alpha$, $\beta$, and $\gamma$) using the `train_parameters` function. Then, the code will output the associated matrix generated from `get_matrix`.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `0.2` absolute error tolerance), the output will be `"Correct!"`. Otherwise, you will receive a `"Wrong answer"` prompt.

Good luck!

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

np.random.seed(1967)


In [2]:
def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    
    alpha, beta, gamma, phi = params
    
    mx = np.exp(phi*1j) * qml.matrix( qml.Rot(alpha, beta, gamma, wires=0) )
    
    return mx


def error(U, params):
    """
    This function determines the similarity between your generated matrix and the target unitary.

    Args:
        - U (matrix): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = np.real(get_matrix(params))
    return np.linalg.norm(matrix - U)
    

In [3]:
def train_parameters(U):

    epochs = 1000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)

    return params


In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    return json.dumps(params)

def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)


In [5]:
test_cases = [['[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'], ['[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
Correct!
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
Correct!
